In [1]:
import os
import cv2

import numpy as np

import matplotlib.pyplot as plt

import matplotlib.image as mpimg

import open3d as o3d

from utils import readXML
import copy

In [21]:
class Intrinsics:
    def __init__(self, xml_dir):
        self.xml_dir = xml_dir

    
    def from_camera(self, camera="020122061233", H=640, W=480):
        intrinsic_pth = os.path.join(self.xml_dir, f"{camera}.xml")
        xmldict = readXML(intrinsic_pth)
        params = [float(param) for param in xmldict['camera']['camera_model']['params'][3:-3].split(";")]
        
        fx_old, fy_old, cx, cy, *_ = params
        
        dimx_old = int(xmldict['camera']['camera_model']['width'])
        dimy_old = int(xmldict['camera']['camera_model']['height'])
        dimx_new, dimy_new = H, W
        
        fx_new = (dimx_new / dimx_old) * fx_old
        fy_new = (dimy_new / dimy_old) * fy_old
        
        intrinsic = o3d.camera.PinholeCameraIntrinsic(W, H, fx_new, fy_new, cx, cy)
        
        return intrinsic

class Extrinsics:
    def __init__(self, xml_dir, default_camera="020122061233"):
        self.xml_dir = xml_dir
        self.default_camera = default_camera
        
    def from_camera(self, camera="020122061233"):
        if camera == self.default_camera:
            return np.array([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]])
        
        extrinsic_pth = os.path.join(self.xml_dir, f"{self.default_camera}-{camera}.xml")
        xmldict = readXML(extrinsic_pth)
        T_wc = [param.split(",") for param in xmldict['camera']['pose']['T_wc'][3:-3].split(";")]
        T_wc.append([0.,0.,0.,1.])
        T_wc = np.array(T_wc)
        T_wc = T_wc.astype(np.float64)
        
        return T_wc
    
default_extrinsic=np.array([[1., 0., 0., 0.], [0., 1., 0., 0.], [0., 0., 1., 0.], [0., 0., 0., 1.]])

In [6]:
def get_rgbd(color_pth, depth_pth, cam_scale = 1):
    depth = mpimg.imread(depth_pth)
    color = mpimg.imread(color_pth)
    
    img = o3d.geometry.Image(color.astype(np.uint8))
    depth = np.asarray(depth).astype(np.float32) / cam_scale
    depth = o3d.geometry.Image(depth)

    rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(img, depth)
    
    return rgbd

class RGBD:
    def __init__(self, recordings_dir, recording="20210709_004128"):
        self.recordings_dir = recordings_dir
        self.recording = recording
        
    def from_camera(self, camera="020122061233", idx="000000"):
        color_name = f"color_{idx}.jpg"
        depth_name = f"aligned_depth_to_color_{idx}.png"

        color_pth = os.path.join(self.recordings_dir, self.recording, camera, color_name)
        depth_pth = os.path.join(self.recordings_dir, self.recording, camera, depth_name)
        
        rgbd = get_rgbd(color_pth, depth_pth)
        
        return rgbd

In [11]:
xml_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/"

intrinsics = Intrinsics(xml_dir)
extrinsics = Extrinsics(xml_dir)
rgbds = RGBD(xml_dir, recording="20210709_004128")

# Img 020122061233

In [22]:
camera = "020122061233"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_1 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)

o3d.visualization.draw_geometries([pcd_1])

# camera 821312060044

In [47]:
camera = "821312060044"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

# Remove translation
# extrinsic[:,-1] = extrinsic[:,-1]/100_000
extrinsic[:,-1] = np.array([0,0,0,1])

pcd_2 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)

In [48]:
o3d.visualization.draw_geometries([pcd_1, pcd_2])

Same but with o3d rotation and translation

In [83]:
camera = "821312060044"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_2 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, default_extrinsic)

rotation = extrinsic[:3,:3]
translation = extrinsic[:3,-1]/100_000 *np.array([640,480,1])/np.array([1280,720,1])
pcd_2_r = copy.deepcopy(pcd_2).rotate(rotation)
pcd_2_r = copy.deepcopy(pcd_2_r).translate(-t)


o3d.visualization.draw_geometries([pcd_1, pcd_2_r])

In [61]:
t*100_000

array([[ 0.44410217],
       [-0.68084482],
       [ 0.37925991]])

In [60]:
extrinsic[:3,-1]

array([ 0.4200817 , -0.1460006 ,  0.07875547])

In [64]:
translation = extrinsic[:3,-1]/100_000 *np.array([640,480,1])/np.array([1280,720,1])
translation

array([ 2.10040850e-06, -9.73337333e-07,  7.87554700e-07])

# img 020122061651

In [ ]:
camera = "020122061651"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

extrinsic[:,-1] = np.array([0,0,0,1])

pcd_3 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)

In [ ]:
o3d.visualization.draw_geometries([pcd_3, pcd_2])

# Find transformation

In [ ]:
source = pcd_2
target = pcd

In [ ]:
threshold = 500
trans_init = np.asarray(
            [[1, 0, 0,  0],
            [0, 1, 0,  0],
            [0, 0,  1, 0],
            [0, 0, 0, 1]])

target.estimate_normals()
reg_p2l = o3d.pipelines.registration.registration_icp(source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration = 50))


reg_p2p = o3d.pipelines.registration.registration_icp(source, target, threshold, trans_init,
            o3d.pipelines.registration.TransformationEstimationPointToPoint(),
            o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration = 1))

In [ ]:
source_temp = copy.deepcopy(source)
source_temp.transform(reg_p2p.transformation)
source_temp.paint_uniform_color([1, 0.0, 0])

o3d.visualization.draw_geometries([target, source])

In [ ]:
option = o3d.pipelines.odometry.OdometryOption()
odo_init = np.identity(4)
print(option)

[success_color_term, trans_color_term,
 info] = o3d.pipelines.odometry.compute_rgbd_odometry(
     source_rgbd_image, target_rgbd_image, intrinsic, odo_init,
     o3d.pipelines.odometry.RGBDOdometryJacobianFromColorTerm(), option)
[success_hybrid_term, trans_hybrid_term,
 info] = o3d.pipelines.odometry.compute_rgbd_odometry(
     source_rgbd_image, target_rgbd_image, intrinsic, odo_init,
     o3d.pipelines.odometry.RGBDOdometryJacobianFromHybridTerm(), option)

In [ ]:
trans_color_term

In [ ]:
trans_hybrid_term

In [54]:
def rigid_transform_3D(A, B):
    # Code from https://github.com/nghiaho12/rigid_transform_3D/blob/master/rigid_transform_3D.py
    assert A.shape == B.shape

    num_rows, num_cols = A.shape
    if num_rows != 3:
        raise Exception(f"matrix A is not 3xN, it is {num_rows}x{num_cols}")

    num_rows, num_cols = B.shape
    if num_rows != 3:
        raise Exception(f"matrix B is not 3xN, it is {num_rows}x{num_cols}")

    # find mean column wise
    centroid_A = np.mean(A, axis=1)
    centroid_B = np.mean(B, axis=1)

    # ensure centroids are 3x1
    centroid_A = centroid_A.reshape(-1, 1)
    centroid_B = centroid_B.reshape(-1, 1)

    # subtract mean
    Am = A - centroid_A
    Bm = B - centroid_B

    H = Am @ np.transpose(Bm)

    # sanity check
    #if linalg.matrix_rank(H) < 3:
    #    raise ValueError("rank of H = {}, expecting 3".format(linalg.matrix_rank(H)))

    # find rotation
    U, S, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # special reflection case
    if np.linalg.det(R) < 0:
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    t = -R @ centroid_A + centroid_B

    return R, t

In [71]:
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, 'C:/Users/lucas/Desktop/UPC/MIT/manopth/manus/')

from extract_points import pick_points



In [75]:
pick_points(pcd_2_r)


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) Afther picking points, press q for close the window
[Open3D INFO] Picked point #186525 (-1.4e-06, -3.8e-06, 9.7e-06) to add in queue.
[Open3D INFO] Picked point #118380 (-4.5e-06, -2.7e-06, 8.8e-06) to add in queue.
[Open3D INFO] Picked point #70654 (-5.2e-06, -5.3e-06, 1.1e-05) to add in queue.
[Open3D INFO] Picked point #119218 (-2.1e-06, -6.5e-06, 1.1e-05) to add in queue.



[186525, 118380, 70654, 119218]

In [76]:
# pcd
picked_ids = [94170, 115176, 25254, 14250]

# pcd_2
picked_ids = [190850, 117292, 70117, 119751]

# Nothing
picked_ids = [186525, 118380, 70654, 119218]


point_cloud = np.asarray(pcd_2_r.points)
keypoint_3d_position = []
for point_id in picked_ids:
    point_in_world = point_cloud[point_id, :]
    keypoint_3d_position.append([float(point_in_world[0]), float(point_in_world[1]), float(point_in_world[2])])
    
keypoint_3d_position

[[-1.423675622334962e-06, -3.841659167738579e-06, 9.747522976159103e-06],
 [-4.547270025957597e-06, -2.661879392568063e-06, 8.841482962669476e-06],
 [-5.18532776526374e-06, -5.273664188210205e-06, 1.0580175828938093e-05],
 [-2.091597828993573e-06, -6.457796685630581e-06, 1.1485708384882497e-05]]

In [77]:
a = np.array([[-5.279702973355946e-06, 3.5051066306286833e-06, 3.969055899782816e-06],
 [-8.364674713636895e-06, 4.670090167738334e-06, 3.092597090413828e-06],
 [-9.055011864595224e-06, 2.04921663423877e-06, 4.8166512369570385e-06],
 [-5.926625739585855e-06, 8.571089319962642e-07, 5.728753474437406e-06]])

b = np.array([[2.0529716909365156e-07, -2.5224427324230982e-06, 7.721065230725799e-06],
 [-2.845642348979148e-06, -2.0374551155664612e-06, 7.6142523539601825e-06],
 [-4.841492815655481e-06, -4.945709300428062e-06, 9.384299119119532e-06],
 [-1.8773521321918388e-06, -5.410598903015469e-06, 9.55214818532113e-06]])

c = np.array([[2.0529716909365156e-07, -2.5351206812498807e-06, 7.721065230725799e-06],
 [-2.82283155188375e-06, -2.107939174445023e-06, 7.553216164524201e-06],
 [-4.841492815655481e-06, -4.945709300428062e-06, 9.384299119119532e-06],
 [-1.8773521321918388e-06, -5.410598903015469e-06, 9.55214818532113e-06]])

d = np.array([[-1.423675622334962e-06, -3.841659167738579e-06, 9.747522976159103e-06],
 [-4.547270025957597e-06, -2.661879392568063e-06, 8.841482962669476e-06],
 [-5.18532776526374e-06, -5.273664188210205e-06, 1.0580175828938093e-05],
 [-2.091597828993573e-06, -6.457796685630581e-06, 1.1485708384882497e-05]])

In [ ]:
pcd_pts = np.array([[-8.186385772121419e-06, -2.870964911048856e-06, 1.0086213478643913e-05],
 [-5.450979665026556e-06, -1.7795954795095105e-06, 8.087282367341686e-06],
 [-4.1039848097672424e-05, -1.613264668335712e-05, 3.305104110040702e-05],
 [-6.334644569122416e-06, -6.860979353496837e-06, 1.31837950902991e-05]])

pcd_2_pts = np.array([[-3.0709718109808933e-06, 4.13122200967874e-08, 6.851300895505119e-06],
 [-5.875839644683174e-06, -1.7143199349005031e-06, 8.026246177905705e-06],
 [-3.465911598876267e-06, -3.3203956331541428e-06, 9.292743925470859e-06],
 [-6.512043048454424e-07, -1.6751177386491631e-06, 8.087282367341686e-06]])

In [55]:
R, t = rigid_transform_3D(pcd_pts.T, pcd_2_pts.T)

NameError: name 'pcd_pts' is not defined

In [78]:
R, t = rigid_transform_3D(c.T, d.T)

R

array([[ 0.99669267, -0.07837276, -0.02148084],
       [ 0.0791915 ,  0.99604194,  0.04036295],
       [ 0.01823246, -0.04193056,  0.99895416]])

In [79]:
t

array([[-1.09575927e-06],
       [-9.84120972e-07],
       [ 1.50530847e-06]])

In [ ]:
np.linalg.inv(R)

In [ ]:
t

In [ ]:
np.hstack([R,t])

In [ ]:
T_wc

In [ ]:
np.array(keypoint_3d_position)*1000000

In [ ]:
a = (np.array(keypoint_3d_position)[0] + np.array(keypoint_3d_position)[1])/2

In [ ]:
b = (np.array(keypoint_3d_position)[2] + np.array(keypoint_3d_position)[3])/2

In [ ]:
(a+b)/2

In [ ]:
a

In [ ]:
b

In [ ]:
np.array(keypoint_3d_position)*1000000